# Material and Energy Balance

In [4]:
import pandas as pd
import scipy
from eosHSG import getCompound

In [15]:
if False:
    methane = getCompound("Methane","srk")
    T = 300 #K
    P = 1e6 #Pa
    Z = 1 
    print methane.G(T, P, Z)/1e6  #Hooray!


## Lets make as simple Pandas Dataframe

In [20]:
df_simple = pd.DataFrame({"Cities":["Gondor","Umbar","Numenor","Mordor"],
                          "Population":[20000, 10000, 100000, 500000],
                          "IsEvil":[False, True, True, True],
                          "Ruler":["Denethor","Corsair","Ar-Pharazon","Sauron"]},
                        index = ["Gnd","Umb","Num","Mor"])
df_simple

,Cities,IsEvil,Population,Ruler
Gnd,Gondor,False,20000,Denethor
Umb,Umbar,True,10000,Corsair
Num,Numenor,True,100000,Ar-Pharazon
Mor,Mordor,True,500000,Sauron


In [22]:
#A pandas dataframe is a cross between a numpy array and a python dictionary.
df_simple.loc["Gnd"]  #like a dictionary

Cities          Gondor
IsEvil           False
Population       20000
Ruler         Denethor
Name: Gnd, dtype: object

In [23]:
df_simple.loc["Gnd"]["Cities"] #like a dictionary

'Gondor'

In [25]:
df_simple["Cities"] + df_simple["Ruler"]  #like an array

Gnd        GondorDenethor
Umb          UmbarCorsair
Num    NumenorAr-Pharazon
Mor          MordorSauron
dtype: object

## Making a Process Stream

In [32]:
#Lets use 5 compounds: Nitrogen, Oxygen, Water, Carbon-Dioxide and Methane
typeeos = 'pr'
methane = getCompound("Methane", typeeos)
nitrogen = getCompound("Nitrogen", typeeos)
oxygen = getCompound("Oxygen", typeeos)
carbondioxide = getCompound("Carbon dioxide",typeeos)
water = getCompound("Water",typeeos)
dict_compounds = {"CH4":methane,
                 "N2":nitrogen,
                 "O2":oxygen,
                 "CO2":carbondioxide,
                 "H2O":water}  
#dict_compounds gives us access to all the properties of each compound

Ok.  What properties of each stream are useful to us?
<ol>
<li> Temperature K </li>
<li> Pressure Pa </li>
<li> State of the Steam (Liquid or Vapour) (we won't be bothering with mixed streams just yet) </li>
<li> Molar Flow Rates of Each Component kmol/s</li>
<li> Mol fractions of each component </li>
<li> Total Molar flow rate kmol/s </li>
<li> Total Mass Flow rate kg/s </li>
<li> Total Enthalpy in J/s </li>
<li> Total Entropy in J/Ks </li>
</ol>
<p> We will use a dataframe for this and you will see how similar this is to Excel.  Lets make the index.

In [128]:
index =  ["T", "P", "isLiq"]
index += ["MolarFlow_%s"%name for name in dict_compounds]
index += ["MolFr_%s"%name for name in dict_compounds]
index += ["TotMolFlow","TotMassFlow","Enthalpy","Entropy"]
index += ["Ncomps"]

columns = ["Air","Fuel"]
df = pd.DataFrame(index=index, columns=columns)
for col in df.columns:
    df.loc["Ncomps"][col] = len(dict_compounds)
df

,Air,Fuel
T,NaN,NaN
P,NaN,NaN
isLiq,NaN,NaN
MolarFlow_N2,NaN,NaN
MolarFlow_CO2,NaN,NaN
MolarFlow_CH4,NaN,NaN
MolarFlow_O2,NaN,NaN
MolarFlow_H2O,NaN,NaN
MolFr_N2,NaN,NaN
MolFr_CO2,NaN,NaN


So its empty.  Duh!  We have to fill it.  

In [131]:
df["Fuel"]["T"] = 300 #K
df["Fuel"]["P"] = 1.013e5 #Pa
df["Fuel"]["MolarFlow_CH4"] = 1.0 #kmol/s
df["Fuel"]["MolarFlow_N2"] = 0.0
df["Fuel"]["MolarFlow_O2"] = 0.0
df["Fuel"]["MolarFlow_CO2"] = 0.0
df["Fuel"]["MolarFlow_H2O"] = 0.0
df["Fuel"]["isLiq"] = False

df["Air"]["T"] = 300 #K
df["Air"]["P"] = 1.013e5 #Pa
df["Air"]["MolarFlow_CH4"] = 0.0 #kmol/s
df["Air"]["MolarFlow_N2"] = 3.0/0.21*0.79
df["Air"]["MolarFlow_O2"] = 3.0
df["Air"]["MolarFlow_CO2"] = 0.0
df["Air"]["MolarFlow_H2O"] = 0.0
df["Air"]["isLiq"] = False

#So what about mol fraction?  We have to write a function.
def get_molfraction(stream):
    ncomps = stream.loc["Ncomps"]
    molflow = stream.iloc[3:3+ncomps]
    totmolflow = molflow.sum()
    stream.loc["TotMolFlow"] = totmolflow 
    stream.iloc[3+ncomps:3+2*ncomps] = (stream.iloc[3:3+ncomps]/stream.iloc[3:3+ncomps].sum()).values
    return stream


In [132]:
df = df.apply(get_molfraction)
df

,Air,Fuel
T,300,300
P,101300,101300
isLiq,False,False
MolarFlow_N2,11.28571,0
MolarFlow_CO2,0,0
MolarFlow_CH4,0,1
MolarFlow_O2,3,0
MolarFlow_H2O,0,0
MolFr_N2,0.79,0
MolFr_CO2,0,0
